https://climate-change.canada.ca/climate-data/#/hourly-climate-data





https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2010-01-01%2000:00:00/2022-12-31%2000:00:00&CLIMATE_IDENTIFIER=3012206&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=30000

In [ ]:
https://api.weather.gc.ca/collections/climate-hourly/items?datetime=1953-01-01%2000:00:00/2023-01-22%2000:00:00&CLIMATE_IDENTIFIER=3050520&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=140000

In [ ]:
!pip install -q polars
!pip install -q 'black[jupyter]'
!pip install -q pendulum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 KB 20.1 MB/s eta 0:00:00


In [ ]:
!black '/content/drive/MyDrive/Colab Notebooks/Transformers/Download.ipynb'

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import requests
import polars as pl
import requests
from io import StringIO
from tqdm.notebook import tqdm
from time import sleep
import re
import pendulum

We get data from `https://climate.weather.gc.ca/climate_data/bulk_data_e.html`.
We use stations in Alberta to get 12 years of hourly data.

You can get a list of stations [here](https://acis.alberta.ca/weather-data-viewer.jsp).

Hourly metered energy volumes can be found [here](https://www.aeso.ca/market/market-and-system-reporting/data-requests/hourly-metered-volumes-and-pool-price-and-ail-data-2010-to-2022/)


In [ ]:
# Generate start and end dates for each month from 2010 to 2022 using pendulum
def gen_dates(start: str, end: str):
    l = []
    d = pendulum.parse(start).date()
    while d <= pendulum.parse(end).date():
        l.append(
            {"start_of_month": d.start_of("month"), "end_of_month": d.end_of("month")}
        )
        d = d.add(months=1)
    return l


dates = gen_dates("2010-01-01", "2022-12-31")
dates[:5]

[{'start_of_month': Date(2010, 1, 1), 'end_of_month': Date(2010, 1, 31)},
 {'start_of_month': Date(2010, 2, 1), 'end_of_month': Date(2010, 2, 28)},
 {'start_of_month': Date(2010, 3, 1), 'end_of_month': Date(2010, 3, 31)},
 {'start_of_month': Date(2010, 4, 1), 'end_of_month': Date(2010, 4, 30)},
 {'start_of_month': Date(2010, 5, 1), 'end_of_month': Date(2010, 5, 31)}]

In [ ]:
def gen_urls():
    climate_id = [
        "3031094",  # Calgary
        "3012206",  # Edmonton
        "3033890",  # Lethbridge
        "3062696",  # Fort McMurray
        "3035208",  # Pincher Creek
        "3026KNQ",  # Sundre A
    ]
    urls = []
    for station in climate_id:
        for date in gen_dates("2010-01-01", "2022-12-31"):
            # url = (
            #     "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?"
            #     + "format=csv"
            #     + f"&stationID={station}&Year={year}&Month={month}&timeframe=1"
            #     + "&submit= Download+Data"
            # )
            # url = ("https://climate.weather.gc.ca/climate_data/bulk_data_e.html?" +
            #        f"format=csv&stationID={station}&Year={year}&Month={month}" +
            #        "&Day=20&time=LST&timeframe=1&submit=Download+Data")
            url = (
                "https://api.weather.gc.ca/collections/climate-hourly/items?"
                + f"""datetime={date["start_of_month"]}%2000:00:00/{date["end_of_month"]}%2000:00:00"""
                + f"""&CLIMATE_IDENTIFIER={station}"""
                + "&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE"
                + "&f=csv&limit=10000&startindex=0"
            )
            urls.append(url)
    return urls


urls = gen_urls()
urls[0:4]

['https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2010-01-01%2000:00:00/2010-01-31%2000:00:00&CLIMATE_IDENTIFIER=3031094&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0',
 'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2010-02-01%2000:00:00/2010-02-28%2000:00:00&CLIMATE_IDENTIFIER=3031094&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0',
 'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2010-03-01%2000:00:00/2010-03-31%2000:00:00&CLIMATE_IDENTIFIER=3031094&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0',
 'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2010-04-01%2000:00:00/2010-04-30%2000:00:00&CLIMATE_IDENTIFIER=3031094&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0']

In [ ]:
def get_data(urls):
    dfs = []
    fails = []
    for url in tqdm(urls):
        try:
            df = pl.read_csv(url, infer_schema_length=0)
            dfs.append(df)
        except Exception as e:

            fails.append({"url": url, "error": e})
            continue

    return dfs, fails

In [ ]:
dfs, fails = get_data(urls)

  0%|          | 0/936 [00:00<?, ?it/s]

In [ ]:
df = pl.concat(dfs)

In [ ]:
tdf = df.clone()
tdf.head()

x,y,STATION_NAME,CLIMATE_IDENTIFIER,ID,LOCAL_DATE,PROVINCE_CODE,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,LOCAL_HOUR,TEMP,TEMP_FLAG,DEW_POINT_TEMP,DEW_POINT_TEMP_FLAG,HUMIDEX,HUMIDEX_FLAG,PRECIP_AMOUNT,PRECIP_AMOUNT_FLAG,RELATIVE_HUMIDITY,RELATIVE_HUMIDITY_FLAG,STATION_PRESSURE,STATION_PRESSURE_FLAG,VISIBILITY,VISIBILITY_FLAG,WINDCHILL,WINDCHILL_FLAG,WIND_DIRECTION,WIND_DIRECTION_FLAG,WIND_SPEED,WIND_SPEED_FLAG
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""-114.000297222...","""51.10944722222...","""CALGARY INT'L ...","""3031094""","""3031094.2010.1...","""2010-01-01 00:...","""AB""","""2010""","""1""","""1""","""0""","""-21.6""",null,"""-23.9""",null,null,null,null,null,"""82""",null,"""89.38""",null,null,null,null,null,null,"""M""",null,"""M"""
"""-114.000297222...","""51.10944722222...","""CALGARY INT'L ...","""3031094""","""3031094.2010.1...","""2010-01-01 01:...","""AB""","""2010""","""1""","""1""","""1""","""-21.2""",null,"""-23.5""",null,null,null,null,null,"""82""",null,"""89.25""",null,null,null,null,null,null,"""M""",null,"""M"""
"""-114.000297222...","""51.10944722222...","""CALGARY INT'L ...","""3031094""","""3031094.2010.1...","""2010-01-01 02:...","""AB""","""2010""","""1""","""1""","""2""","""-20.8""",null,"""-23.0""",null,null,null,null,null,"""82""",null,"""89.21""",null,null,null,null,null,null,"""M""",null,"""M"""
"""-114.000297222...","""51.10944722222...","""CALGARY INT'L ...","""3031094""","""3031094.2010.1...","""2010-01-01 03:...","""AB""","""2010""","""1""","""1""","""3""","""-20.4""",null,"""-22.6""",null,null,null,null,null,"""83""",null,"""89.12""",null,null,null,null,null,null,"""M""",null,"""M"""
"""-114.000297222...","""51.10944722222...","""CALGARY INT'L ...","""3031094""","""3031094.2010.1...","""2010-01-01 04:...","""AB""","""2010""","""1""","""1""","""4""","""-20.4""",null,"""-22.7""",null,null,null,null,null,"""82""",null,"""89.04""",null,null,null,null,null,null,"""M""",null,"""M"""


In [ ]:
tdf = tdf.with_columns(
    [
        pl.col("x").cast(pl.Float64),
        pl.col("y").cast(pl.Float64),
        pl.col("LOCAL_DATE").str.strptime(pl.Datetime, fmt="%Y-%m-%d %T"),
        pl.col("LOCAL_YEAR").cast(pl.Int64),
        pl.col("LOCAL_MONTH").cast(pl.Int64),
        pl.col("LOCAL_DAY").cast(pl.Int64),
        pl.col("LOCAL_HOUR").cast(pl.Int64),
        pl.col("TEMP").cast(pl.Float64),
        pl.col("DEW_POINT_TEMP").cast(pl.Float64),
        pl.col("HUMIDEX").cast(pl.Int64),
        pl.col("PRECIP_AMOUNT").cast(pl.Float64),
        pl.col("RELATIVE_HUMIDITY").cast(pl.Int64),
        pl.col("STATION_PRESSURE").cast(pl.Float64),
        pl.col("WINDCHILL").cast(pl.Int64),
        pl.col("WIND_DIRECTION").cast(pl.Int64),
        pl.col("WIND_SPEED").cast(pl.Int64),
    ]
)

In [107]:
tdf.columns = [i.lower() for i in tdf.columns]
tdf.head()

x,y,station_name,climate_identifier,id,local_date,province_code,local_year,local_month,local_day,local_hour,temp,temp_flag,dew_point_temp,dew_point_temp_flag,humidex,humidex_flag,precip_amount,precip_amount_flag,relative_humidity,relative_humidity_flag,station_pressure,station_pressure_flag,visibility,visibility_flag,windchill,windchill_flag,wind_direction,wind_direction_flag,wind_speed,wind_speed_flag
f64,f64,str,str,str,datetime[μs],str,i64,i64,i64,i64,f64,str,f64,str,i64,str,f64,str,i64,str,f64,str,str,str,i64,str,i64,str,i64,str
-114.000297,51.109447,"""CALGARY INT'L ...","""3031094""","""3031094.2010.1...",2010-01-01 00:00:00,"""AB""",2010,1,1,0,-21.6,null,-23.9,null,null,null,null,null,82,null,89.38,null,null,null,null,null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L ...","""3031094""","""3031094.2010.1...",2010-01-01 01:00:00,"""AB""",2010,1,1,1,-21.2,null,-23.5,null,null,null,null,null,82,null,89.25,null,null,null,null,null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L ...","""3031094""","""3031094.2010.1...",2010-01-01 02:00:00,"""AB""",2010,1,1,2,-20.8,null,-23.0,null,null,null,null,null,82,null,89.21,null,null,null,null,null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L ...","""3031094""","""3031094.2010.1...",2010-01-01 03:00:00,"""AB""",2010,1,1,3,-20.4,null,-22.6,null,null,null,null,null,83,null,89.12,null,null,null,null,null,null,"""M""",null,"""M"""
-114.000297,51.109447,"""CALGARY INT'L ...","""3031094""","""3031094.2010.1...",2010-01-01 04:00:00,"""AB""",2010,1,1,4,-20.4,null,-22.7,null,null,null,null,null,82,null,89.04,null,null,null,null,null,null,"""M""",null,"""M"""


In [108]:
tdf.write_csv(
    "/content/drive/MyDrive/Colab Notebooks/Transformers/energy/ab_weather.csv"
)

In [ ]:
energy = pl.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/Transformers/energy/ab_energy.csv",
    infer_schema_length=111744,
)
energy.head()

Date_Begin_GMT,Date_Begin_Local,AFG1,AKE1,ALP1,ALP2,APS1,BCR2,BCRK,BIG,BOW1,BR3,BR4,BR5,BRA,BTR1,CAL1,CHIN,CMH1,CNR5,CR1,CRS1,CRS2,CRS3,CRW1,DAI1,DKSN,DOWG,DRW1,DV1,EAGL,EC01,EC04,ENC1,ENC2,ENC3,FNG1,...,SET1,WEF1,WRW1,HYS1,JER1,WHT2,ERV2,HRT1,RTL1,TVS1,NPC3,STR1,STR2,BRK1,BRK2,COL1,NMK1,MON1,CRD1,CRD2,WHE1,KKP1,KKP2,FMG1,ERV3,SRL1,EPS1,HHW1,ACTUAL_POOL_PRICE,ACTUAL_AIL,DAY_AHEAD_POOL_PRICE,EXPORT_BC,EXPORT_MT,EXPORT_SK,IMPORT_BC,IMPORT_MT,IMPORT_SK
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,...,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,i64,i64,f64,i64
"""1/1/2010 7:00""","""1/1/2010 0:00""",5.2305,0.0,0.0,0.0,0.0,11.8714,0.0,26.573744,42.754208,149.157,156.2886,383.1201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.394,4.7568,126.860491,0.0,7.1244,24.437387,40.6058,19.116,0.0,0.0,0.0,13.05024,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,51.12,8307,49.68,125,null,0,50,null,44
"""1/1/2010 8:00""","""1/1/2010 1:00""",5.236,0.0,0.0,0.0,0.0,11.858,0.0,26.753066,42.066657,149.7589,155.4531,382.9625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.856,4.7568,118.03819,0.0,6.0588,23.965469,25.0991,18.8,0.0,0.0,0.0,13.27632,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,48.79,8186,38.0,125,null,0,0,null,45
"""1/1/2010 9:00""","""1/1/2010 2:00""",5.5605,0.0,0.0,0.0,0.0,11.8566,0.0,26.544753,45.459836,149.2844,151.495,376.4841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.226,4.7568,84.194438,0.0,7.29,24.471967,8.5649,18.716,0.0,0.0,0.0,13.37832,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,39.56,8075,44.45,150,null,0,50,null,45
"""1/1/2010 10:00...","""1/1/2010 3:00""",5.625,0.0,0.0,0.0,0.0,11.8467,0.0,26.826099,38.234323,149.4653,151.2895,374.1055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.072,4.7544,80.459613,0.0,7.542,24.379821,1.2963,17.76,0.0,0.0,0.0,13.61136,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,36.27,8013,36.21,150,null,0,50,null,44
"""1/1/2010 11:00...","""1/1/2010 4:00""",7.1375,0.0,0.0,0.0,0.0,11.8596,0.0,26.861537,44.831311,150.0822,151.4089,374.8381,0.0,0.1113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.226,4.7688,80.432869,0.0,7.2828,24.599399,1.2996,16.876,0.0,0.0,0.0,13.85232,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,36.16,7982,36.24,100,null,0,0,null,95


In [ ]:
# df.columns = [i.lower() for i in df.columns]
energy.columns = [
    (
        re.sub("\(.+?\)", "", i.lower()).strip().replace(" ", "_").replace("/", "_")
    ).replace(".", "")
    for i in energy.columns
]

energy = energy[[s.name for s in energy if not (s.null_count() == energy.height)]]

In [ ]:
energy.head()

date_begin_gmt,date_begin_local,afg1,ake1,alp1,alp2,aps1,bcr2,bcrk,big,bow1,br3,br4,br5,bra,btr1,cal1,chin,cmh1,cnr5,cr1,crs1,crs2,crs3,crw1,dai1,dksn,dowg,drw1,dv1,eagl,ec01,ec04,enc1,enc2,enc3,fng1,...,set1,wef1,wrw1,hys1,jer1,wht2,erv2,hrt1,rtl1,tvs1,npc3,str1,str2,brk1,brk2,col1,nmk1,mon1,crd1,crd2,whe1,kkp1,kkp2,fmg1,erv3,srl1,eps1,hhw1,actual_pool_price,actual_ail,day_ahead_pool_price,export_bc,export_mt,export_sk,import_bc,import_mt,import_sk
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,...,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,i64,i64,f64,i64
"""1/1/2010 7:00""","""1/1/2010 0:00""",5.2305,0.0,0.0,0.0,0.0,11.8714,0.0,26.573744,42.754208,149.157,156.2886,383.1201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.394,4.7568,126.860491,0.0,7.1244,24.437387,40.6058,19.116,0.0,0.0,0.0,13.05024,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,51.12,8307,49.68,125,null,0,50,null,44
"""1/1/2010 8:00""","""1/1/2010 1:00""",5.236,0.0,0.0,0.0,0.0,11.858,0.0,26.753066,42.066657,149.7589,155.4531,382.9625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.856,4.7568,118.03819,0.0,6.0588,23.965469,25.0991,18.8,0.0,0.0,0.0,13.27632,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,48.79,8186,38.0,125,null,0,0,null,45
"""1/1/2010 9:00""","""1/1/2010 2:00""",5.5605,0.0,0.0,0.0,0.0,11.8566,0.0,26.544753,45.459836,149.2844,151.495,376.4841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.226,4.7568,84.194438,0.0,7.29,24.471967,8.5649,18.716,0.0,0.0,0.0,13.37832,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,39.56,8075,44.45,150,null,0,50,null,45
"""1/1/2010 10:00...","""1/1/2010 3:00""",5.625,0.0,0.0,0.0,0.0,11.8467,0.0,26.826099,38.234323,149.4653,151.2895,374.1055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.072,4.7544,80.459613,0.0,7.542,24.379821,1.2963,17.76,0.0,0.0,0.0,13.61136,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,36.27,8013,36.21,150,null,0,50,null,44
"""1/1/2010 11:00...","""1/1/2010 4:00""",7.1375,0.0,0.0,0.0,0.0,11.8596,0.0,26.861537,44.831311,150.0822,151.4089,374.8381,0.0,0.1113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.226,4.7688,80.432869,0.0,7.2828,24.599399,1.2996,16.876,0.0,0.0,0.0,13.85232,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,36.16,7982,36.24,100,null,0,0,null,95


In [ ]:
energy.write_csv(
    "/content/drive/MyDrive/Colab Notebooks/Transformers/energy/ab_energy.csv"
)